In [1]:
#%cd ./final/

In [2]:
!samtools view -b -o CLIP-chr5.bam -F 16 ../binfo1-datapack1/CLIP-35L33G.bam chr5
!samtools view CLIP-chr5.bam | wc -l

814346


In [4]:
!samtools view CLIP-chr5.bam > CLIP-chr5.txt
#!head -20 CLIP-chr5.txt

In [5]:
!samtools mpileup CLIP-chr5.bam > CLIP-chr5.pileup
!wc -l CLIP-chr3.pileup

[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000
20466330 CLIP-chr3.pileup


In [1]:
import pandas as pd
import numpy as np
import re
import collections
import math

In [7]:
def estimate_shannon_entropy(sequence):
    m = len(sequence)
    bases = collections.Counter([tmp_base for tmp_base in sequence])
    shannon_entropy_value = 0
    for base in bases:
        # number of residues
        n_i = bases[base]
        # n_i (# residues type i) / M (# residues in column)
        p_i = n_i / float(m)
        entropy_i = p_i * (math.log(p_i, 2))
        shannon_entropy_value += entropy_i
 
    return shannon_entropy_value * -1

In [8]:
pileup = pd.read_csv('CLIP-chr5.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])

toremove = re.compile('[<>$*#^!]')
pileup['matches'] = pileup['basereads'].apply(lambda x: toremove.sub('', x))
pileup[['chrom', 'pos', 'matches']].head(20)

,chrom,pos,matches
0,chr5,3063186,GGIGIG
1,chr5,3063187,AAA
2,chr5,3063188,AAA
3,chr5,3063189,AAA
4,chr5,3063190,AAA
5,chr5,3063191,AAA
6,chr5,3063192,AAA
7,chr5,3063193,AAA
8,chr5,3063194,GGG
9,chr5,3063195,GGG


In [9]:
value = []
end_pos = []
for i in range(len(pileup)) :
    tmp = estimate_shannon_entropy(pileup['matches'][i])
    value.append(tmp)
    end_pos.append(pileup['pos'][i]+1)

In [10]:
chr = pileup['chrom']
start_pos = pileup['pos']
bedform = {'chr':chr, 'start_pos' : start_pos, 'end_pos' : end_pos, 'value' : value }
bedform = pd.DataFrame(bedform)
bedform.head()

,chr,start_pos,end_pos,value
0,chr5,3063186,3063187,0.918296
1,chr5,3063187,3063188,-0.000000
2,chr5,3063188,3063189,-0.000000
3,chr5,3063189,3063190,-0.000000
4,chr5,3063190,3063191,-0.000000


In [11]:
bedform.to_csv("./bedform_chr5.txt", sep="\t", index=False)

In [12]:
high_entropy = bedform[bedform['value'] > 0.8]
high_entropy.head()
high_entropy.to_csv("./bedfrom_high_entropy_chr5.txt", sep="\t", index=False)

In [13]:
!samtools depth CLIP-chr5.bam > CLIP-chr5_depth.depth

In [14]:
!awk '$3>=50 {print $1"\t"$2"\t"$3}' CLIP-chr5_depth.depth > CLIP-chr5_depth.txt

In [15]:
depths = pd.read_csv('CLIP-chr5_depth.txt', sep='\t', names=['chrom', 'pos', 'depth'])
depths.tail

<bound method NDFrame.tail of       chrom        pos  depth
0      chr5    3286627    213
1      chr5    3286628    229
2      chr5    3286629    229
3      chr5    3286630    229
4      chr5    3286631    229
...     ...        ...    ...
55593  chr5  150731439     91
55594  chr5  150731440     75
55595  chr5  150731441     61
55596  chr5  150731442     61
55597  chr5  150731443     54

[55598 rows x 3 columns]>

In [16]:
high_depth_pos = depths['pos']
high_depth_pos

0          3286627
1          3286628
2          3286629
3          3286630
4          3286631
           ...    
55593    150731439
55594    150731440
55595    150731441
55596    150731442
55597    150731443
Name: pos, Length: 55598, dtype: int64

In [17]:
hehd = pd.DataFrame()
for pos in high_depth_pos :
    tmp = high_entropy[high_entropy['start_pos']==pos]
    if len(tmp) != 0 :
        hehd = pd.concat([hehd,tmp])

In [18]:
len(hehd)
hehd.head()

,chr,start_pos,end_pos,value
650,chr5,3286627,3286628,0.980092
659,chr5,3286636,3286637,1.441961
660,chr5,3286637,3286638,1.127404
691,chr5,3286668,3286669,0.839403
715,chr5,3286692,3286693,1.069764


In [19]:
chr = hehd['chr']
start_pos = hehd['start_pos']
end_pos = hehd['end_pos']
input_bedtools = {'chr':chr, 'start_pos' : start_pos, 'end_pos' : end_pos, 'name' : hehd.chr+ hehd.start_pos.map(str), 'strand' : "+" }
input_bedtools = pd.DataFrame(input_bedtools)
input_bedtools.tail()
input_bedtools.to_csv("./input_bedtools_chr5.bed", sep="\t", index=False, header=False)

In [20]:
!bedtools slop -i input_bedtools_chr5.bed -g hg38.genome -b 15 > chr5_binding_region.bed

In [21]:
!head -5 chr5.fa

>chr5
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN


In [22]:
!bedtools getfasta -fi chr5.fa -bed chr5_binding_region.bed -fo chr5_binding_region.fa

index file chr5.fa.fai not found, generating...


In [2]:
from Bio import motifs
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

In [24]:
input_file = 'chr5_binding_region.fa'
#Read fastq file and store it as a pd dataframe
with open(input_file, 'r') as f:
    data = f.readlines()
#     print(data)

In [25]:
data = [line.strip('\n') for line in data]
# print(data)

In [26]:
seq_id = [data[i] for i in range(len(data))if i%2 == 0]
sequence = [Seq(data[i].upper()) for i in range(len(data))if i%2 == 1] 

In [27]:
print(len(sequence))
print(len(sequence[1]))
print(seq_id[1])
print(sequence[1])
print(str(sequence[1])[2])

4587
31
>chr5:3286621-3286652
TCATTCATTCATTCAGTGAATATCTGCCCAG
A


In [28]:
select_seq = []
for idx in range(0,len(sequence)) :
    tmp_id = seq_id[idx]
    tmp_seq = sequence[idx]
    mid = str(tmp_seq)[13:16]
    last = str(tmp_seq)[17]
    if mid == "AAG" and last == "G":
        tmp_seq = Seq(re.sub('T',"U",str(tmp_seq)))
        rec = SeqRecord(tmp_seq,id=tmp_id)
        select_seq.append(rec)

In [29]:
SeqIO.write(select_seq, "select_AAGNGH_chr5.fa", "fasta")

10

In [30]:
select_seq = []
for idx in range(0,len(sequence)) :
    tmp_id = seq_id[idx]
    tmp_seq = sequence[idx]
    mid = str(tmp_seq)[13:16]
    last = str(tmp_seq)[18]
    if mid == "AAG" and last == "G":
        tmp_seq = Seq(re.sub('T',"U",str(tmp_seq)))
        rec = SeqRecord(tmp_seq,id=tmp_id)
        select_seq.append(rec)

In [31]:
SeqIO.write(select_seq, "select_AAGNHG_chr5.fa", "fasta")

18

In [ ]:
!cat select_AAGNHG_* > select_AAGNHG.fa

In [ ]:
!cat select_AAGNGH_* > select_AAGNGH.fa

In [24]:
!cat *_binding_region.fa > binding_region.fa

In [25]:
input_file = 'binding_region.fa'
#Read fastq file and store it as a pd dataframe
with open(input_file, 'r') as f:
    data = f.readlines()
#     print(data)

In [26]:
data = [line.strip('\n') for line in data]
seq_id = [data[i] for i in range(len(data))if i%2 == 0]
sequence = [Seq(data[i].upper()) for i in range(len(data))if i%2 == 1] 

In [27]:
sequence[475]

Seq('GAAAAAAGTAGACAAGGGAAGAAAGCCAGAA')

In [29]:
select_seq = []
for idx in range(0,len(sequence)) :
    tmp_id = seq_id[idx]
    tmp_seq = sequence[idx]
    mid = str(tmp_seq)[15]
    if mid == "G" :
        tmp_seq = Seq(re.sub('T',"U",str(tmp_seq)))
        rec = SeqRecord(tmp_seq[10:21],id=tmp_id)
        select_seq.append(rec)

In [30]:
select_seq[0]

SeqRecord(seq=Seq('GACAAGGGAAG'), id='>chr13:18994599-18994630', name='<unknown name>', description='<unknown description>', dbxrefs=[])

In [31]:
SeqIO.write(select_seq, "binding_region.fa", "fasta")

6548